## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import load_partition,save_partition, create_train_valid_loaders, RandomTransform

from purrfect.training import train_model
import torch.optim as optim

from purrfect.active_learning import create_new_partition, test_model

from sklearn.model_selection import train_test_split
#from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32*2
USE_AUTOCAST = True

## Definición modelo

In [4]:
class GradientMagnitude(nn.Module):
    def __init__(self):
        super(GradientMagnitude, self).__init__()
        # Define Sobel filters for computing gradients in x and y directions
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32,device=DEVICE)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32,device=DEVICE)
        
        # Sobel filters need to be reshaped for convolution: (out_channels, in_channels, height, width)
        # Apply the filters across all channels by expanding them to shape (C, 1, 3, 3)
        self.sobel_x = sobel_x.view(1, 1, 3, 3)
        self.sobel_y = sobel_y.view(1, 1, 3, 3)
    
    def forward(self, x):
        B, C, W, H = x.shape
        
        # Apply Sobel filters to compute gradients in x and y directions for all channels
        grad_x = F.conv2d(x, self.sobel_x.expand(C, 1, 3, 3), groups=C, padding=1)
        grad_y = F.conv2d(x, self.sobel_y.expand(C, 1, 3, 3), groups=C, padding=1)
        
        # Compute gradient magnitude: sqrt(grad_x^2 + grad_y^2)
        grad_mag = torch.sqrt(grad_x**2 + grad_y**2)
        
        return grad_mag
class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        self.grad_magnitude = GradientMagnitude()

    def forward(self, x):
        
        # Extract the first, second, and third channels
        e1 = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        e2 = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        delta = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        magnitude = torch.sqrt(e1**2 + e2**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        angle = 0.5 * torch.atan(e1 / e2 )

        e1_weighted = e1 / magnitude
        e2_weighted = e2 / magnitude

        cross_product = e1 * e2


        


        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        """ grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2) """

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        #e1_weighted = first_channel / (third_channel + epsilon)
        #e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            e1,
            e2,
            delta,
            magnitude,
            angle,
            e1_weighted,
            e2_weighted,
            cross_product
        ], dim=1)
        output = torch.cat([output, self.grad_magnitude(output)], dim=1)

        return output

In [5]:
BN_MOMENTUM = 0.1
def kaiming_init(module,
                 a=0,
                 mode='fan_out',
                 nonlinearity='relu',
                 bias=0,
                 distribution='normal'):
    assert distribution in ['uniform', 'normal']
    if distribution == 'uniform':
        nn.init.kaiming_uniform_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    else:
        nn.init.kaiming_normal_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    if hasattr(module, 'bias') and module.bias is not None:
        nn.init.constant_(module.bias, bias)
class PA(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=1, stride=1):
        super(PSA_s, self).__init__()

        self.inplanes = inplanes
        self.inter_planes = planes // 2
        self.planes = planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = (kernel_size - 1) // 2
        ratio = 4

        self.conv_q_right = nn.Conv2d(self.inplanes, 1, kernel_size=1, stride=stride, padding=0, bias=False)
        self.conv_v_right = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                      bias=False)
        # self.conv_up = nn.Conv2d(self.inter_planes, self.planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv_up = nn.Sequential(
            nn.Conv2d(self.inter_planes, self.inter_planes // ratio, kernel_size=1),
            nn.LayerNorm([self.inter_planes // ratio, 1, 1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.inter_planes // ratio, self.planes, kernel_size=1)
        )
        self.softmax_right = nn.Softmax(dim=2)
        self.sigmoid = nn.Sigmoid()

        self.conv_q_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # g
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_v_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # theta
        self.softmax_left = nn.Softmax(dim=2)

        self.reset_parameters()
    def reset_parameters(self):
        kaiming_init(self.conv_q_right, mode='fan_in')
        kaiming_init(self.conv_v_right, mode='fan_in')
        kaiming_init(self.conv_q_left, mode='fan_in')
        kaiming_init(self.conv_v_left, mode='fan_in')

        self.conv_q_right.inited = True
        self.conv_v_right.inited = True
        self.conv_q_left.inited = True
        self.conv_v_left.inited = True

    def spatial_pool_mask(self, x):
        input_x = self.conv_v_right(x)

        batch, channel, height, width = input_x.size()

        # [N, IC, H*W]
        input_x = input_x.view(batch, channel, height * width)

        # [N, 1, H, W]
        context_mask = self.conv_q_right(x)

        # [N, 1, H*W]
        context_mask = context_mask.view(batch, 1, height * width)

        # [N, 1, H*W]
        context_mask = self.softmax_right(context_mask)

        # [N, IC, 1]
        # context = torch.einsum('ndw,new->nde', input_x, context_mask)
        context = torch.matmul(input_x, context_mask.transpose(1, 2))

        # [N, IC, 1, 1]
        context = context.unsqueeze(-1)

        # [N, OC, 1, 1]
        context = self.conv_up(context)

        # [N, OC, 1, 1]
        mask_ch = self.sigmoid(context)
        return mask_ch

    def channel_pool_mask(self, x):
        # [N, IC, H, W]
        g_x = self.conv_q_left(x)

        batch, channel, height, width = g_x.size()

        # [N, IC, 1, 1]
        avg_x = self.avg_pool(g_x)

        batch, channel, avg_x_h, avg_x_w = avg_x.size()

        # [N, 1, IC]
        avg_x = avg_x.view(batch, channel, avg_x_h * avg_x_w).permute(0, 2, 1)

        # [N, IC, H*W]
        theta_x = self.conv_v_left(x).view(batch, self.inter_planes, height * width)

        # [N, IC, H*W]
        theta_x = self.softmax_left(theta_x)

        # [N, 1, H*W]
        # context = torch.einsum('nde,new->ndw', avg_x, theta_x)
        context = torch.matmul(avg_x, theta_x)

        # [N, 1, H, W]
        context = context.view(batch, 1, height, width)

        # [N, 1, H, W]
        mask_sp = self.sigmoid(context)
        return mask_sp
    def forward(self, x, y):
        sp_mask = self.spatial_pool_mask(y)
        ch_mask = self.channel_pool_mask(y)
        return x * sp_mask * ch_mask
    

class PSA_s(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=1, stride=1):
        super(PSA_s, self).__init__()

        self.inplanes = inplanes
        self.inter_planes = planes // 2
        self.planes = planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = (kernel_size - 1) // 2
        ratio = 4

        self.conv_q_right = nn.Conv2d(self.inplanes, 1, kernel_size=1, stride=stride, padding=0, bias=False)
        self.conv_v_right = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                      bias=False)
        # self.conv_up = nn.Conv2d(self.inter_planes, self.planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv_up = nn.Sequential(
            nn.Conv2d(self.inter_planes, self.inter_planes // ratio, kernel_size=1),
            nn.LayerNorm([self.inter_planes // ratio, 1, 1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.inter_planes // ratio, self.planes, kernel_size=1)
        )
        self.softmax_right = nn.Softmax(dim=2)
        self.sigmoid = nn.Sigmoid()

        self.conv_q_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # g
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_v_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # theta
        self.softmax_left = nn.Softmax(dim=2)

        self.reset_parameters()

    def reset_parameters(self):
        kaiming_init(self.conv_q_right, mode='fan_in')
        kaiming_init(self.conv_v_right, mode='fan_in')
        kaiming_init(self.conv_q_left, mode='fan_in')
        kaiming_init(self.conv_v_left, mode='fan_in')

        self.conv_q_right.inited = True
        self.conv_v_right.inited = True
        self.conv_q_left.inited = True
        self.conv_v_left.inited = True

    def spatial_pool(self, x):
        input_x = self.conv_v_right(x)

        batch, channel, height, width = input_x.size()

        # [N, IC, H*W]
        input_x = input_x.view(batch, channel, height * width)

        # [N, 1, H, W]
        context_mask = self.conv_q_right(x)

        # [N, 1, H*W]
        context_mask = context_mask.view(batch, 1, height * width)

        # [N, 1, H*W]
        context_mask = self.softmax_right(context_mask)

        # [N, IC, 1]
        # context = torch.einsum('ndw,new->nde', input_x, context_mask)
        context = torch.matmul(input_x, context_mask.transpose(1, 2))

        # [N, IC, 1, 1]
        context = context.unsqueeze(-1)

        # [N, OC, 1, 1]
        context = self.conv_up(context)

        # [N, OC, 1, 1]
        mask_ch = self.sigmoid(context)

        out = x * mask_ch

        return out

    def channel_pool(self, x):
        # [N, IC, H, W]
        g_x = self.conv_q_left(x)

        batch, channel, height, width = g_x.size()

        # [N, IC, 1, 1]
        avg_x = self.avg_pool(g_x)

        batch, channel, avg_x_h, avg_x_w = avg_x.size()

        # [N, 1, IC]
        avg_x = avg_x.view(batch, channel, avg_x_h * avg_x_w).permute(0, 2, 1)

        # [N, IC, H*W]
        theta_x = self.conv_v_left(x).view(batch, self.inter_planes, height * width)

        # [N, IC, H*W]
        theta_x = self.softmax_left(theta_x)

        # [N, 1, H*W]
        # context = torch.einsum('nde,new->ndw', avg_x, theta_x)
        context = torch.matmul(avg_x, theta_x)

        # [N, 1, H, W]
        context = context.view(batch, 1, height, width)

        # [N, 1, H, W]
        mask_sp = self.sigmoid(context)

        out = x * mask_sp

        return out

    def forward(self, x):
        # [N, C, H, W]
        out = self.spatial_pool(x)

        # [N, C, H, W]
        out = self.channel_pool(out)

        # [N, C, H, W]
        # out = context_spatial + context_channel

        return out

class ConBnRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,bias=False):
        super(ConBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                              dilation=dilation, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out
class DoubleConv(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, use_attention=True):
        super(DoubleConv, self).__init__()
        if use_attention:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
                PSA_s(out_channels, out_channels),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
            )
        else:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
            )

        self.rescale = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        return self.seq(x) + self.rescale(x)


# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.embedder = nn.Sequential(
            DoubleConv(in_channels, 16), # in_channelsx128x128 -> 16x128x128
            DoubleConv(16, 16), # 16x128x128 -> 16x128x128
        )
        # Encoder
        self.encoder1 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.AvgPool2d(2), # 16x128x128 -> 16x64x64
            DoubleConv(16, 32) # in_channelsx64x64 -> 32x64x64
        )
        
        self.encoder2 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.AvgPool2d(2), # 32x64x64 -> 32x32x32
            DoubleConv(32, 64) # 32x32x32 -> 64x32x32
        )
        
        self.encoder3 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.AvgPool2d(2), # 64x32x32 -> 64x16x16
            DoubleConv(64, 128) # 64x16x16 -> 128x16x16
        )
        self.encoder4 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.AvgPool2d(2), # 128x16x16 -> 128x8x8
            DoubleConv(128, 256) # 128x8x8 -> 256x8x8
        )

        
        self.upconv4 = nn.Sequential(
            #nn.Dropout2d(0.1),
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2) # 256x8x8 -> 128x16x16
        )
        
        # concat 128x16x16 + 128x16x16 -> 256x16x16
        self.decoder4 = DoubleConv(256, 128) # 256x16x16 -> 128x16x16

        self.upconv3 = nn.Sequential(
            #nn.Dropout2d(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2) # 128x16x16 -> 64x32x32
        )
        # concat 64x32x32 + 64x32x32 -> 128x32x32
        self.decoder3 = DoubleConv(128, 64) # 128x32x32 -> 64x32x32

        self.upconv2 = nn.Sequential(
            #nn.Dropout2d(0.2),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2) # 64x32x32 -> 32x64x64
        )
        # concat 32x64x64 + 32x64x64 -> 64x64x64
        self.decoder2 = DoubleConv(64, 32) # 64x64x64 -> 32x64x64

        self.upconv1 = nn.Sequential(
            #nn.Dropout2d(0.3),
            nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2) # 32x64x64 -> 16x128x128
        )
        # concat 16x128x128 + 16x128x128 -> 32x128x128
        self.decoder1 = DoubleConv(32, 16) # 32x128x128 -> 16x128x128

        self.output = nn.Conv2d(16, out_channels, kernel_size=1) # 16x128x128 -> out_channelsx128x128

    def forward(self, x):
        x = self.embedder(x)
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        #Decoder with skip connections
        d4 = self.upconv4(e4)
        d4 = torch.cat((e3, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((x, d1), dim=1)
        d1 = self.decoder1(d1)

        out = self.output(d1)
        return out
class Magnitude(nn.Module):
    def __init__(self, in_channels):
        super(Magnitude, self).__init__()
        self.conv = nn.Sequential(
            DoubleConv(in_channels, 16, use_attention=False), # -> 16x128x128
            nn.AvgPool2d(4), # -> 16x32x32
            DoubleConv(16, 8, use_attention=False), # -> 8x32x32
            nn.AvgPool2d(4), # -> 8x8x8
            nn.Conv2d(8, 1, kernel_size=8) # -> 1x1x1
        )
    def forward(self, x):
        return  self.conv(x)
class KappaPredictor(nn.Module):
    def __init__(self):
        super(KappaPredictor, self).__init__()
        total_channels = 16
        self.channel_adder = ChannelAdder()
        self.unet = nn.Sequential(
            nn.BatchNorm2d(total_channels),
            UNet(total_channels, 1)
        )
    def forward(self, x):
        x = self.channel_adder(x)
        return self.unet(x)
        
        

## Creación particion inicial

In [6]:
def create_next_partitions(current_partition,k=1):#Creacion de particiones train y valid
    init_partition = []
    for i in range(k):
        init_partition += load_partition(f"partition_{k*current_partition+(i+1)}.json")
        #print(f"partition_{k*current_partition+(i+1)}.json")

    train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
    save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
    save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

In [7]:
create_next_partitions(0,1)

## Carga modelo

In [10]:
#Define model
model = KappaPredictor().to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()
transform = RandomTransform()

In [11]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

2222955

In [12]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)

In [13]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=BATCH_SIZE,
    transform=transform,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.AdamW(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=5,
    use_autocast=USE_AUTOCAST
)

Epoch [1/50]


Train Epoch 1:   0%|          | 0/77 [00:00<?, ?it/s]

Validate Epoch 1: 100%|██████████| 20/20 [00:05<00:00,  3.57it/s, WMAPE=4.3, DICE=0.532, DPEAKS=217, Loss=0.0269]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 20/20 [00:05<00:00,  3.68it/s, WMAPE=6.21, DICE=0.446, DPEAKS=190, Loss=0.024]


...

Epoch [24/50]


Validate Epoch 24: 100%|██████████| 20/20 [00:05<00:00,  3.67it/s, WMAPE=9.51, DICE=0.177, DPEAKS=95.7, Loss=0.0126]


Epoch [25/50]


Validate Epoch 25: 100%|██████████| 20/20 [00:05<00:00,  3.66it/s, WMAPE=9.5, DICE=0.181, DPEAKS=99.4, Loss=0.024]

early stopping: 5 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)

Validate Epoch test: 100%|██████████| 486/486 [02:33<00:00,  3.16it/s, WMAPE=4.7, DICE=0.263, DPEAKS=125, Loss=0.0169]


In [12]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)

Validate Epoch test: 100%|██████████| 486/486 [02:33<00:00,  3.18it/s, WMAPE=3.2, DICE=0.137, DPEAKS=90, Loss=0.0106]


In [13]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)

Validate Epoch test: 100%|██████████| 486/486 [02:36<00:00,  3.11it/s, WMAPE=3.1, DICE=0.118, DPEAKS=66.9, Loss=0.00923]


In [11]:
create_next_partitions(current_partition)

In [9]:
for current_partition in range(1,20):
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=BATCH_SIZE,
    )
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    optimizer = optim.AdamW(model.parameters())
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=5,
    )
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
    create_next_partitions(current_partition)


Epoch [32/50]


Validate Epoch 32: 100%|██████████| 20/20 [00:05<00:00,  3.85it/s, WMAPE=4.49, DICE=0.113, DPEAKS=90.2, Loss=0.0129]


Epoch [33/50]


Validate Epoch 33: 100%|██████████| 20/20 [00:05<00:00,  3.97it/s, WMAPE=2.92, DICE=0.113, DPEAKS=84.9, Loss=0.0102]


Saving best model
Epoch [34/50]


...

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 20/20 [00:06<00:00,  3.24it/s, WMAPE=3.93, DICE=0.0853, DPEAKS=87.2, Loss=0.0117]


Saving best model
Epoch [2/50]


Train Epoch 2:  96%|█████████▌| 74/77 [00:40<00:01,  1.79it/s, loss=0.00669]

KeyboardInterrupt: 

In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
optimizer = optim.Adam(model.parameters())
#optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 971/971 [03:04<00:00,  5.28it/s, WMAPE=1.82, DICE=0.0536, DPEAKS=39, Loss=0.00555]


In [ ]:
for current_partition in range(3,9):
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=BATCH_SIZE,
    )
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=3,
    )
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    optimizer = optim.Adam(model.parameters())
    optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
    test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
    create_next_partitions(current_partition)

In [19]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [20]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  5.96it/s, WMAPE=2.03, DICE=0.0603, DPEAKS=61.1, Loss=0.00608]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.13it/s, WMAPE=5.36, DICE=0.0579, DPEAKS=45, Loss=0.0157]


Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.04, DICE=0.0634, DPEAKS=56.6, Loss=0.00611]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.73, DICE=0.0758, DPEAKS=71.2, Loss=0.0079]


early stopping: 3 epochs without improvement
Training complete.


In [21]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:54<00:00,  6.59it/s, WMAPE=1.77, DICE=0.0499, DPEAKS=38.4, Loss=0.00553]


TypeError: create_next_partitions() missing 2 required positional arguments: 'model' and 'criterion'

In [23]:
create_next_partitions(current_partition)

In [24]:
current_partition=4

In [25]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.06it/s, WMAPE=1.51, DICE=0.0393, DPEAKS=27.1, Loss=0.00439]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.91it/s, WMAPE=1.72, DICE=0.0389, DPEAKS=28.4, Loss=0.00493]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:10<00:00,  7.07it/s, WMAPE=1.45, DICE=0.0379, DPEAKS=27.2, Loss=0.00424]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.50it/s, WMAPE=1.41, DICE=0.038, DPEAKS=26.9, Loss=0.00424]


early stopping: 3 epochs without improvement
Training complete.


In [26]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 1941/1941 [04:50<00:00,  6.68it/s, WMAPE=1.41, DICE=0.0383, DPEAKS=28.1, Loss=0.00421]


In [27]:
current_partition=5

In [10]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  7.86it/s, WMAPE=1.87, DICE=0.0453, DPEAKS=46.4, Loss=0.00513]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.59it/s, WMAPE=1.97, DICE=0.0453, DPEAKS=44, Loss=0.00521]


Epoch [3/50]


Validate Epoch 3: 100%|██████████| 77/77 [00:09<00:00,  8.01it/s, WMAPE=2.45, DICE=0.0461, DPEAKS=44, Loss=0.00671]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:09<00:00,  7.87it/s, WMAPE=1.92, DICE=0.0455, DPEAKS=45.8, Loss=0.00514]


early stopping: 3 epochs without improvement
Training complete.


In [11]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:10<00:00,  7.21it/s, WMAPE=1.65, DICE=0.0436, DPEAKS=36.1, Loss=0.00477]


In [12]:
current_partition=6

In [13]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  8.14it/s, WMAPE=2.05, DICE=0.046, DPEAKS=52.4, Loss=0.00538]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.04it/s, WMAPE=2.06, DICE=0.0462, DPEAKS=52.8, Loss=0.00534]


Saving best model
Epoch [3/50]


...

Epoch [14/50]


Validate Epoch 14: 100%|██████████| 77/77 [00:09<00:00,  8.05it/s, WMAPE=2.14, DICE=0.0452, DPEAKS=49.8, Loss=0.00559]


Epoch [15/50]


Validate Epoch 15: 100%|██████████| 77/77 [00:09<00:00,  8.12it/s, WMAPE=2.04, DICE=0.046, DPEAKS=50.1, Loss=0.00524]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_7_val: 100%|██████████| 77/77 [00:10<00:00,  7.23it/s, WMAPE=1.54, DICE=0.043, DPEAKS=33.5, Loss=0.00457]
